In [20]:
import sqlite3
import json
import pandas as pd
import polyline
import networkx as nx
import numpy as np
from sklearn.neighbors import BallTree
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from datetime import datetime

# need this to activate: 
# source venvPy13/bin/activate


In [21]:
small_ids = [
    12292106042,
    8287258968,
    4380620390,
    14226775715,
    9715432255,
    7496473660,
]

# --- Connect to DB ---
conn = sqlite3.connect("strava_cache.db")
cursor = conn.cursor()
cursor.execute("SELECT id, data FROM activities")
rows = cursor.fetchall()

activity_coords = {}  # act_id -> list of (lat, lon)
# --- 4. Loop through activities and draw each PolyLine ---
for activity_id, data_json in rows:
    data = json.loads(data_json)
    poly = data.get("map", {}).get("summary_polyline")
    if not poly:
        continue

    coords = polyline.decode(poly)
    if coords:
        activity_coords[activity_id] = coords
act_array = np.array(list(activity_coords.keys()))


pt_list = []
id_list = []
for act_id in act_array:
    coords = activity_coords[act_id]
    for c in coords:
        pt_rad = np.radians(c)
        pt_list.append(pt_rad)
        id_list.append(act_id)

# pt_array = np.array(pt_list)
# id_array = np.array(id_list)

# all_pts = {} # map point id to point coords (rad and ll)
# for aid, coords in activity_coords.items():
#     for c in range(len(coords)):
#         pt_rad = np.radians(coords[c])
#         pt_id = f'{aid}_{c}'
#         all_pts[pt_id] = pt_rad
# all_pt_ids = list(all_pts.keys())
# pt_array = np.array([all_pts[pt_id] for pt_id in all_pt_ids])  # lat, lon in radians



In [22]:
pt_array = np.array(pt_list)
id_array = np.array(id_list)
print(pt_array.shape)
print(id_array.shape)


(611475, 2)
(611475,)


In [23]:
# --- 5. Build graph ---
act_G = nx.Graph()

# Add nodes
for act_id in act_array:
    act_G.add_node(act_id)
print(f"act Graph has {act_G.number_of_nodes()} nodes")


# # --- 5. Build graph ---
# pt_G = nx.Graph()

# # Add nodes
# for pt_id in all_pt_ids:
#     pt_G.add_node(pt_id)
# print(f"pt Graph has {pt_G.number_of_nodes()} nodes")

act Graph has 2272 nodes


In [24]:
# Define BallTree using centroids in radians
tree = BallTree(pt_array, metric='haversine')
# Define the search radius (400 meters = 0.4 km) in radians
radius_km = 0.1
radius_radians = radius_km / 6371.0088  # Earth's radius in km
all_neighbors = tree.query_radius(pt_array, r=radius_radians)

# takes around 2 min @ 0.4
# takes around 1.5 min @ 0.5

In [ ]:
t1 = datetime.now()
act_neighbors = {}
for pt_ind in range(len(pt_array)):
    
    act_id = id_array[pt_ind]
    if act_id not in act_neighbors:
        act_neighbors[act_id] = set()
    else:
        act_neighbors[act_id].add(act_id)

    nes = all_neighbors[pt_ind]
    for n_ind in nes:
        n_id = id_array[n_ind]
        act_neighbors[act_id].add(n_id)
    if pt_ind % 10000 == 0:
        t2 = datetime.now()
        print(f"Processed {pt_ind} points... in {t2-t1}")
        t1 = t2

# takes around 35 min @ 0.4
# takes around 81 min @ 0.5
# takes around 5min @ 0.1
# why does the time per 10000 pts increase? Maybe I need to figure out how to cache component numbers then just add a single activity 

Processed 0 points... in 0:00:00.004946
Processed 10000 points... in 0:00:01.531205
Processed 20000 points... in 0:00:01.285642
Processed 30000 points... in 0:00:01.287254
Processed 40000 points... in 0:00:00.639960
Processed 50000 points... in 0:00:00.553805
Processed 60000 points... in 0:00:00.584374
Processed 70000 points... in 0:00:00.996453
Processed 80000 points... in 0:00:01.568639
Processed 90000 points... in 0:00:01.330713
Processed 100000 points... in 0:00:00.469613
Processed 110000 points... in 0:00:01.175610
Processed 120000 points... in 0:00:03.574420
Processed 130000 points... in 0:00:02.925716
Processed 140000 points... in 0:00:00.892458
Processed 150000 points... in 0:00:01.806508
Processed 160000 points... in 0:00:02.562309
Processed 170000 points... in 0:00:10.687677
Processed 180000 points... in 0:00:15.819414
Processed 190000 points... in 0:00:02.650527
Processed 200000 points... in 0:00:04.181874
Processed 210000 points... in 0:00:08.715022
Processed 220000 points.

In [ ]:
for act_id, nes in act_neighbors.items():
    for neighbor_id in nes:
        if act_id != neighbor_id:
            act_G.add_edge(act_id, neighbor_id)
    
print(f"Graph has {act_G.number_of_nodes()} nodes and {act_G.number_of_edges()} edges.")
components = list(nx.connected_components(act_G))
print(f'{len(components)} connected components')

merged = components

# first run with 0.4 - missing some CT segments
# Graph has 2272 nodes and 113606 edges.
# 259 connected components

# second run with 0.5 - better on CT segments
# Graph has 2272 nodes and 115926 edges.
# 247 connected components

# third run with 0.1
# Graph has 2272 nodes and 100493 edges.
# 291 connected components

Graph has 2272 nodes and 100493 edges.
291 connected components


In [30]:
# Output: merged list of act_id groups
checked_ids = {}
for comp_num in range(len(merged)):
    for act_id in merged[comp_num]:
        checked_ids[act_id] = comp_num

colormap = cm.get_cmap('tab20', len(merged))
component_colors = {}

for c_ind in range(len(merged)):
    
    component_colors[c_ind] = colors.to_hex(colormap(c_ind))

m = folium.Map(location=coords[-1], zoom_start=12,tiles="Cartodb Positron")

# --- 3. Draw each activity polyline ---
for aid, cc_color_code in checked_ids.items():
    folium.PolyLine(
        activity_coords[aid],
        color=component_colors.get(cc_color_code, "#888"),
        weight=3,
        opacity=0.7,
        popup=f"color {cc_color_code}"
    ).add_to(m)

# --- 5. Save map ---
m.save("cc_map.html")
print("Map saved to cc_map.html")

/var/folders/h6/5v65kvyj4_v_2p32lqgmvws40000gn/T/ipykernel_4498/3453508436.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab20', len(merged))


Map saved to cc_map.html


In [ ]:
act_neighbors = {}

for act_ind in range(len(act_array)):
    self_act_id = act_array[act_ind]
    nes = all_neighbors[act_ind]
    act_neighbors[self_act_id] = set()
    for i in nes:
        pt = pts_list[i]
        act_id = int(pt.split('_')[0])
        if act_id != self_act_id:
            act_neighbors[self_act_id].add(act_id)

    if act_ind % 1000 == 0:
        print(f"Processed {act_ind} activities...")


In [116]:
target_act = 14226775715
target_ind = np.where(act_array==target_act)
pt_nes = all_neighbors[target_ind][0]
for pt_ind in pt_nes:
    pt = all_pt_ids[pt_ind]
    print(pt)


8332443748_185
8332443748_184
8332443748_183
8332443748_182
8332443748_181
3196106570_263
3196106493_93
4351504404_304
3196106570_46
3254566449_286
4505307070_177
3520258656_82
3973925225_69
3480259907_76
3480260645_75
3973926880_143
4477230999_96
4477230999_95
3196106493_70
3196106570_262
3973926880_144
3960265198_46
3196106498_31
3960266289_62
5490568827_288
3196106498_54
5490568827_289
3973926880_119
3196106498_57
3196106498_55
3196106498_56
3531361299_122
3531361299_121
3531361299_120
3254566449_157
5490568827_290
3254566449_266
3973925225_92
3489236754_283
3489236754_284
3489236754_285
3254566449_262
3531361299_123
3973925225_91
3480259907_94
3973926880_118
3973926880_124
3254566449_265
3254566449_153
3254566449_264
3480259907_96
3254566449_154
3254566449_263
3254566449_155
3531361299_124
3254566449_156
3480259907_95
3531361299_119
3973925225_93
3480259907_93
5517653989_276
3973926880_123
3480259907_92
3973926880_120
3973926880_121
5517653989_275
5517653989_274
3973926880_122
3196

In [58]:
for act_id, nes in act_neighbors.items():
    for neighbor_id in nes:
        if act_id != neighbor_id:
            act_G.add_edge(act_id, neighbor_id)
    
print(f"Graph has {act_G.number_of_nodes()} nodes and {act_G.number_of_edges()} edges.")
components = list(nx.connected_components(act_G))
print(f'{len(components)} connected components')

Graph has 2272 nodes and 134144 edges.
1 connected components


In [ ]:
check_ind = 2025
self_act_id = act_array[check_ind]
print(f'{self_act_id},')
for i in act_neighbors[self_act_id]:
    print(f'{i},')

In [45]:
n_nes = []
for aid, nes in act_neighbors.items():
    n_nes.append(len(nes))
w = np.array(n_nes)
w.argmax()

np.int64(2025)

In [16]:
t1 = datetime.now()
for i in range(10000000):
    i
t2 = datetime.now()
print(t2-t1)

0:00:01.196483
